In [ ]:
import pickle, glob, os

In [ ]:
docs = {}
product = []
service = []
count = {}

for filename in glob.glob('*.pickle'):
   with open(os.path.join(os.getcwd(), filename), 'rb') as f:
     pick_company = pickle.load(f)
     company = filename.split(".")[0]
     count[company] = {}
     for key,value in pick_company.items():
       
       if company in ['Google','Amazon','Cisco','Microsoft','IBM','Oracle','Goldman Sachs','Oracle','Juniper','Western Digital','Schneider','Intel','Siemens']:
         product.extend(value)
       else:
         service.extend(value)
      
       
       if key not in docs:
         docs[key] = []

       
       count[company][key] = 0

       docs[key].extend(value)   
       

       count[company][key] += len(value)
    
       
#for key,value in docs.items():
#  print(key," : ",len(value)) 

print(docs.keys())
print("Product:",len(product))
print("Service:",len(service))

industry = []
industry.extend(product)
industry.extend(service)
print("Total:", len(industry))

print(len(count.keys()))


dict_keys(['Data Scientist', 'Software Engineer', 'Data Analyst', 'Business Analyst', 'Quality Assurance Engineer', 'Front-end Developer', 'Network Engineer', 'Systems Engineer'])
Product: 4901
Service: 2722
Total: 7623
24


In [ ]:
import csv

with open('output.csv', 'wb') as output:
    writer = csv.writer(output)
    for key, value in count.items():
        writer.writerow([key, value])

# Processing

In [ ]:
import pandas as pd
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
cleaned_T = []
cleaned_P = []
cleaned_S = []
for doc in industry:
  cleaned_T.append(remove_stopwords(doc))
for doc in product:
  cleaned_P.append(remove_stopwords(doc))
for doc in service:
  cleaned_S.append(remove_stopwords(doc))



In [ ]:
def get_coherence(model, topics, docs):
  # Preprocess Documents
  documents = pd.DataFrame({"Document": docs,
                            "ID": range(len(docs)),
                            "Topic": topics})
  documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
  cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

  # Extract vectorizer and analyzer from BERTopic
  vectorizer = model.vectorizer_model
  analyzer = vectorizer.build_analyzer()

  # Extract features for Topic Coherence evaluation
  words = vectorizer.get_feature_names()
  tokens = [analyzer(doc) for doc in cleaned_docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]
  topic_words = [[words for words, _ in model.get_topic(topic)] 
                for topic in range(len(set(topics))-1)]

  # Evaluate
  coherence_model = CoherenceModel(topics=topic_words, 
                                  texts=tokens, 
                                  corpus=corpus,
                                  dictionary=dictionary, 
                                  coherence='c_v')
  coherence = coherence_model.get_coherence()
  return coherence


In [ ]:
from bertopic import BERTopic

In [ ]:
model_ind = BERTopic(min_topic_size=5,top_n_words=20,nr_topics="auto")
reduced_t, _ = model_ind.fit_transform(cleaned_T)


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
v_545 = model_ind

In [ ]:
v_545.get_topic_info()

,Topic,Count,Name
0,-1,675,-1_amazon_accenture_wwwaccenturecom_machine
1,0,186,0_google_googles_eeo_visatravel
2,1,164,1_module_coach_guide_junior
3,2,161,2_deloitte_wellbeing_tips_deloittes
4,3,121,3_cisco_learns_ciscos_collaboration
...,...,...,...
275,293,6,293_bsf_cndsr_5g_nrf
276,294,6,294_sharepoint_ood_a1744913_antifraud
277,295,6,295_voc_analyst_artifacts_format
297,296,5,296_deduping_discrepancies_interpreting_accoun...


In [ ]:
coherence_ind_all = {}
topics_ind_all={}

In [ ]:
topics_ind_all["300"] = reduced_t

In [ ]:
coherence_ind_all["300"] = get_coherence(v_545,topics_ind_all["300"],cleaned_T)

In [ ]:
print(coherence_ind_all)

{'300': 0.6717969094477065}


In [ ]:
v_545.save("Ind_300_5x10_autored")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
reduced_t = v_545.reduce_topics(cleaned_T,topics_ind_all["300"],nr_topics=60)

In [ ]:
v_545.get_topic_info()

,Topic,Count,Name
0,-1,931,-1_accenture_management_services_skills
1,0,250,0_andltbr_insights_analysis_analytics
2,1,221,1_aws_amazon_etl_cloud
3,2,204,2_google_eeo_visatravel_restrictive
4,3,197,3_siemens_future_shape_countries
...,...,...,...
56,55,56,55_storage_digital_flash_memory
57,56,55,56_advertising_ad_deloitte_marketing
58,57,55,57_cloud_devsecops_devops_development
59,58,54,58_oracle_customer_fit_knowledge


In [ ]:
v_545.get_topics()

{-1: [('accenture', 0.008811098410884535),
  ('management', 0.007258349413520908),
  ('services', 0.006960307682655166),
  ('skills', 0.006400383764158881),
  ('role', 0.006396679381870973),
  ('clients', 0.006299558043984007),
  ('job', 0.006205058028354729),
  ('requirements', 0.006001729440448202),
  ('customer', 0.005820373489787175),
  ('applications', 0.0057250252371779985)],
 0: [('andltbr', 0.042742351375817694),
  ('insights', 0.029606531890077333),
  ('analysis', 0.019400552756278538),
  ('analytics', 0.01682488468562867),
  ('sql', 0.012504139647929631),
  ('skillsandltbr', 0.011170646865738704),
  ('tech', 0.011027623506008303),
  ('analyst', 0.010014251435072547),
  ('econometrics', 0.00983574340834628),
  ('capabilities', 0.009808285336832328)],
 1: [('aws', 0.018083984735298042),
  ('amazon', 0.014508096645174882),
  ('etl', 0.008510265852832277),
  ('cloud', 0.00843509554236711),
  ('engineer', 0.008304187612849222),
  ('amazons', 0.006836980936290866),
  ('engineering'

In [ ]:
topics_ind_all["60"] = reduced_t[0]

In [ ]:
coherence_ind_all["60"] = get_coherence(v_545,topics_ind_all["60"],cleaned_T)

In [ ]:
print(coherence_ind_all)

{'300': 0.6717969094477065, '60': 0.6068783913475868}


In [ ]:
v_545.visualize_hierarchy(height=1000)

In [ ]:
fig = v_545.visualize_barchart(topics=topics_ind_all["60"],n_words=20,height=20000,width=2000)
fig.write_html("letsee.html")

In [ ]:
with open("60_chart_ind.html", 'a') as plotfile:
  js = "<html><head><script src=\"plotly-2.4.2.min.js\"></script></head><body>"
  plotfile.write(js)
  #for t in topics_ind_all["60"]:
  fig = v_545.visualize_barchart(top_n_topics=20,n_words=20)
  #plotfile.write(fig.to_html(full_html=False, include_plotlyjs=False))
  fig.write_image("uh.pdf",engine="kaleido")


      #plotfile.write("\n\n-------\n\n")  
  #files.download("60_chart_ind.html") 
  files.download("uh.pdf")

In [ ]:
v_545.save("Ind_60_5x10_")

In [ ]:
reduced_t = v_545.reduce_topics(cleaned_T,topics_ind_all["60"],nr_topics=30)

In [ ]:
v_545.get_topic_info()

,Topic,Count,Name
0,-1,1271,-1_skills_security_management_experience
1,0,399,0_amazon_aws_software_customers
2,1,363,1_salesforce_ibm_sap_services
3,2,349,2_oracle_software_java_applications
4,3,338,3_ibm_expertise_business_agile
5,4,303,4_cloud_oracle_services_database
6,5,302,5_network_networking_cisco_routers
7,6,297,6_intel_intels_validation_deposit
8,7,287,7_cisco_collaboration_customer_technologies
9,8,250,8_andltbr_insights_analysis_clients


In [ ]:
v_545.get_topics()

{-1: [('skills', 0.011556410264588361),
  ('security', 0.011389112153493303),
  ('management', 0.011167079955572654),
  ('experience', 0.010583635964779996),
  ('ability', 0.010078990055470657),
  ('services', 0.009794177857814319),
  ('clients', 0.009529350082646033),
  ('development', 0.009280048468725903),
  ('role', 0.00897997270262657),
  ('requirements', 0.008652778197376521)],
 0: [('amazon', 0.027005867638797938),
  ('aws', 0.01498496551639996),
  ('software', 0.013425093790973713),
  ('customers', 0.01293172688379605),
  ('qualifications', 0.012538212160589514),
  ('development', 0.011240316280863553),
  ('services', 0.010782528067482179),
  ('engineer', 0.010640111712132232),
  ('skills', 0.010529325624992316),
  ('java', 0.009770080639196705)],
 1: [('salesforce', 0.025064827120178372),
  ('ibm', 0.01687078978598577),
  ('sap', 0.01246108727864649),
  ('services', 0.010695981212543951),
  ('application', 0.010504028410865458),
  ('expertise', 0.009947885530107795),
  ('requi

In [ ]:
v_545.visualize_hierarchy(height=800)

In [ ]:
v_545.visualize_barchart(n_words=30,top_n_topics=10,height=3000)

In [ ]:
v_545.save("Ind_30_5x10_")
topics_ind_all["30"] = reduced_t[0]
coherence_ind_all["30"] = get_coherence(v_545,topics_ind_all["30"],cleaned_T)
print(coherence_ind_all)

{'300': 0.6717969094477065, '60': 0.6068783913475868, '30': 0.5582427828059361}


In [ ]:
reduced_t = v_545.reduce_topics(cleaned_T,topics_ind_all["30"],nr_topics=15)

In [ ]:
v_545.get_topic_info()

,Topic,Count,Name
0,-1,2020,-1_experience_deloitte_development_skills
1,0,603,0_experience_customers_qualifications_googles
2,1,507,1_network_juniper_networking_software
3,2,463,2_azure_cloud_experience_skills
4,3,454,3_western_digital_experience_storage
5,4,380,4_test_testing_experience_automation
6,5,363,5_microsoft_experience_ibm_knowledge
7,6,351,6_siemens_future_test_software
8,7,349,7_oracle_software_experience_applications
9,8,338,8_ibm_expertise_technology_agile


In [ ]:
v_545.get_topics()

{-1: [('experience', 0.019634270572032965),
  ('deloitte', 0.0181600987768117),
  ('development', 0.017472032351622608),
  ('skills', 0.017280025848718118),
  ('services', 0.013728703020043717),
  ('management', 0.01327241000292284),
  ('clients', 0.012919132933707209),
  ('requirements', 0.010991194842594603),
  ('job', 0.010593326299318608),
  ('professional', 0.010468686896598557)],
 0: [('experience', 0.02769941350202627),
  ('customers', 0.016589030987797914),
  ('qualifications', 0.0161020751206516),
  ('googles', 0.014343105069652516),
  ('cloud', 0.014326894777257146),
  ('engineers', 0.013065894701118129),
  ('engineering', 0.013027894203660424),
  ('engineer', 0.012063490226238545),
  ('skills', 0.011693955526266104),
  ('services', 0.01163920398706887)],
 1: [('network', 0.03742128458342005),
  ('juniper', 0.03643872317539369),
  ('networking', 0.02610105301186293),
  ('software', 0.02101942737314489),
  ('cisco', 0.018086665216039414),
  ('skills', 0.01802372197023807),
  (

In [ ]:
v_545.visualize_hierarchy()

In [ ]:
v_545.save("ind_15_5x10_")
topics_ind_all["15"] = reduced_t[0]
coherence_ind_all["15"] = get_coherence(v_545,topics_ind_all["15"],cleaned_T)
print(coherence_ind_all)

{'300': 0.6717969094477065, '60': 0.6068783913475868, '30': 0.5582427828059361, '15': 0.44418823984965133}


PRODUCT

In [ ]:
model_prod = BERTopic(min_topic_size=5,top_n_words=10,nr_topics="auto")
topics_prod, _ = model_prod.fit_transform(cleaned_P)
coherence_prod = get_coherence(model_prod,topics_prod,cleaned_P)
print(coherence_prod)

0.6811005429005669


In [ ]:
model_prod.get_topic_info()

,Topic,Count,Name
0,-1,412,-1_audit_analytics_aws_sql
1,0,171,0_azure_microsoft_security_365
2,1,147,1_sachs_goldman_financial_banking
3,2,105,2_qa_assurance_automation_testing
4,3,99,3_google_googles_eeo_restrictive
...,...,...,...
179,191,6,191_recommending_apjc_cx_happiness
178,192,6,192_mist_models_mllib_juniper
177,193,6,193_fulfillment_amazon_packing_edams
176,185,6,185_atlassian_appdynamics_j2ee_plugins


In [ ]:
model_prod.save("prod_209_5x10_")
topics_prod_all = {}
topics_prod_all["210"] = topics_prod
coherence_prod_all = {}
coherence_prod_all["210"] = get_coherence(model_prod,topics_prod_all["210"],cleaned_P)
print(coherence_prod_all)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



{'210': 0.6811005429005669}


In [ ]:
reduced_p = model_prod.reduce_topics(cleaned_P,topics_prod_all["210"],nr_topics=60)

In [ ]:
model_prod.get_topic_info()

,Topic,Count,Name
0,-1,439,-1_amazon_management_oracle_skills
1,0,208,0_azure_microsoft_cloud_services
2,1,161,1_sachs_goldman_financial_investment
3,2,152,2_ibm_application_clients_expertise
4,3,145,3_cisco_pim_culture_collaboration
...,...,...,...
56,55,38,55_google_india_googles_cloud
57,56,37,56_intel_ethernet_intels_validation
58,57,37,57_cloud_infrastructure_kubernetes_pipelines
59,58,35,58_tools_intel_api_development


In [ ]:
model_prod.get_topics()

{-1: [('amazon', 0.007614553334033201),
  ('management', 0.0068233762777867805),
  ('oracle', 0.00667451029442974),
  ('skills', 0.0064799748007301275),
  ('qualifications', 0.006135048495607982),
  ('tools', 0.005986156822546476),
  ('job', 0.005722823901157896),
  ('development', 0.005614893224976465),
  ('role', 0.005594169140140558),
  ('knowledge', 0.005425388843644491)],
 0: [('azure', 0.030800422864029283),
  ('microsoft', 0.027466406441756065),
  ('cloud', 0.012191744844288175),
  ('services', 0.010466246507035305),
  ('accommodation', 0.010165304060706554),
  ('protected', 0.009858699586522978),
  ('disability', 0.008517298322151649),
  ('mental', 0.008220879601149504),
  ('platform', 0.008152836217244011),
  ('benefitsperks', 0.008023107628383607)],
 1: [('sachs', 0.03846730702176608),
  ('goldman', 0.03846153928101675),
  ('financial', 0.02756541239627335),
  ('investment', 0.020471405043447043),
  ('management', 0.0197215265125205),
  ('banking', 0.013385587896134),
  ('fir

In [ ]:
# import copy

In [ ]:
# arr_prod = []
# temp = copy.deepcopy(model_prod)
# arr_prod.append({"model":temp,"topics":topics_prod,"topic_count":len(topics_prod)})


In [ ]:
model_prod.save("prod_260_5x10_")
topics_prod_all["60"] = reduced_p[0]
coherence_prod_all["60"] = get_coherence(model_prod,topics_prod_all["60"],cleaned_P)
print(coherence_prod_all)
reduced_p = model_prod.reduce_topics(cleaned_P,topics_prod_all["60"],nr_topics=30)

{'210': 0.6811005429005669, '60': 0.608197249102767}


In [ ]:
model_prod.get_topic_info()

,Topic,Count,Name
0,-1,528,-1_software_skills_experience_management
1,0,293,0_goldman_sachs_financial_firm
2,1,267,1_microsoft_azure_security_services
3,2,257,2_ibm_expertise_application_technology
4,3,227,3_siemens_need_future_test
5,4,203,4_digital_storage_company_flash
6,5,193,5_cisco_collaboration_customer_company
7,6,191,6_learning_ai_models_machine
8,7,173,7_juniper_network_routers_routerswitch
9,8,162,8_intel_intels_issued_deposit


In [ ]:
model_prod.get_topics()

{-1: [('software', 0.011753131738891465),
  ('skills', 0.01162136533343371),
  ('experience', 0.011270291174134412),
  ('management', 0.011049846619843137),
  ('development', 0.009799282887229236),
  ('oracle', 0.00940317268581079),
  ('ability', 0.008882857528969938),
  ('qualifications', 0.008325494079683712),
  ('job', 0.008319782176131163),
  ('engineering', 0.008244452705958454)],
 0: [('goldman', 0.049853388847545366),
  ('sachs', 0.04983465793748764),
  ('financial', 0.0267734789980653),
  ('firm', 0.024830986575067045),
  ('investment', 0.023666198020665514),
  ('clients', 0.019416320758064567),
  ('banking', 0.017976355971360464),
  ('opportunities', 0.01509990239433597),
  ('finance', 0.014971493632349352),
  ('securities', 0.013122499437823417)],
 1: [('microsoft', 0.047694761859473224),
  ('azure', 0.029227160104357493),
  ('security', 0.02455367642657341),
  ('services', 0.01589066752004551),
  ('sap', 0.015460519382900561),
  ('cloud', 0.014574853026210196),
  ('disabilit

In [ ]:
model_prod.visualize_hierarchy(height=800)

In [ ]:
model_prod.save("prod_30_5x10_")
topics_prod_all["30"] = reduced_p[0]
coherence_prod_all["30"] = get_coherence(model_prod,topics_prod_all["30"],cleaned_P)
print(coherence_prod_all)
reduced_p = model_prod.reduce_topics(cleaned_P,topics_prod_all["30"],nr_topics=15)

{'210': 0.6811005429005669, '60': 0.608197249102767, '30': 0.549071874062258}


In [ ]:
model_prod.get_topic_info()

,Topic,Count,Name
0,-1,706,-1_experience_software_oracle_skills
1,0,691,0_ibm_business_expertise_technology
2,1,428,1_siemens_test_testing_experience
3,2,368,2_amazon_experience_software_customers
4,3,331,3_intel_intels_deposit_interviews
5,4,307,4_juniper_software_experience_skills
6,5,293,5_goldman_sachs_financial_investment
7,6,267,6_microsoft_azure_security_services
8,7,238,7_google_googles_experience_eeo
9,8,218,8_cloud_software_experience_oracle


In [ ]:
model_prod.get_topics()

{-1: [('experience', 0.023219819608786607),
  ('software', 0.02013194574687401),
  ('oracle', 0.017949557221072825),
  ('skills', 0.015688275144424474),
  ('development', 0.01453859344632409),
  ('knowledge', 0.012298895918627395),
  ('engineering', 0.012090735242849157),
  ('job', 0.010935624445514368),
  ('applications', 0.01066918501670407),
  ('tools', 0.010362588709957675)],
 0: [('ibm', 0.05195628150873072),
  ('business', 0.030963768032182433),
  ('expertise', 0.025733178070492118),
  ('technology', 0.021531899210744023),
  ('technical', 0.020105843630882976),
  ('experience', 0.017742842731809774),
  ('clients', 0.017225335704972554),
  ('career', 0.014693561329049074),
  ('services', 0.01411846791633413),
  ('cloud', 0.013082781468926193)],
 1: [('siemens', 0.05208879034808216),
  ('test', 0.03858532835940236),
  ('testing', 0.027380746838400728),
  ('experience', 0.024091085485896675),
  ('future', 0.023817869617604983),
  ('automation', 0.02334459207955655),
  ('software', 0

In [ ]:
model_prod.save("prod_15_5x10_")
topics_prod_all["15"] = reduced_p[0]
coherence_prod_all["15"] = get_coherence(model_prod,topics_prod_all["15"],cleaned_P)
print(coherence_prod_all)

{'210': 0.6811005429005669, '60': 0.608197249102767, '30': 0.549071874062258, '15': 0.47400035526341305}


## SERVICE

In [ ]:
model_serv = BERTopic(min_topic_size=5,top_n_words=10,nr_topics="auto")
topics_serv, _ = model_serv.fit_transform(cleaned_S)
coherence_serv = get_coherence(model_serv,topics_serv,cleaned_S)
print(coherence_serv)

0.7011363691472867


In [ ]:
model_serv.get_topic_info()

,Topic,Count,Name
0,-1,287,-1_culture_leadership_deloitte_benefits
1,0,192,0_members_module_coach_mentor
2,1,114,1_test_reporting_defect_traceability
3,2,94,2_alternatives_shortlist_tobeprocesses_diagnosis
4,3,83,3_network_cisco_routers_firewalls
...,...,...,...
98,97,6,97_hosting_operating_ahos_tfo
99,98,6,98_siebel_eai_tata_hiring
100,99,6,99_xamarin_android_ios_sonar
101,100,5,100_ansible_testable_mulesoft_scalable


In [ ]:
model_serv.save("serv_119_5x10_")
topics_serv_all = {}
topics_serv_all["119"] = topics_serv
coherence_serv_all = {}
coherence_serv_all["119"] = coherence_serv #get_coherence(model_serv,topics_serv_all["210"],cleaned_S)
print(coherence_serv_all)

{'119': 0.7011363691472867}


In [ ]:
reduced_s = model_serv.reduce_topics(cleaned_S,topics_serv_all["119"],nr_topics=60)

In [ ]:
model_serv.get_topic_info()

,Topic,Count,Name
0,-1,287,-1_deloitte_support_management_leadership
1,0,192,0_members_module_coach_guide
2,1,130,1_test_reporting_defect_traceability
3,2,115,2_software_engineering_ability_teaming
4,3,94,3_alternatives_shortlist_tobeprocesses_diagnosis
...,...,...,...
56,55,15,55_ai_production_vision_learning
57,56,15,56_scripts_perlshell_airports_solarislinux
58,57,15,57_siebel_tata_consultancy_inmail
59,58,15,58_java_ibm_jdbc_engr


In [ ]:
model_serv.get_topics()

{-1: [('deloitte', 0.008988830670258257),
  ('support', 0.007353282380805939),
  ('management', 0.007254286821380076),
  ('leadership', 0.0068863875418474),
  ('benefits', 0.0061271263664220785),
  ('professionals', 0.006065900314892579),
  ('services', 0.0058791786138613815),
  ('requirements', 0.005847518349797393),
  ('role', 0.005821386187416938),
  ('resources', 0.005657411928345203)],
 0: [('members', 0.06379424129310615),
  ('module', 0.05607307882192415),
  ('coach', 0.04849292560978927),
  ('guide', 0.04703016431227013),
  ('mentor', 0.046686204511224474),
  ('member', 0.045180260545397456),
  ('developer', 0.04071120981483087),
  ('programming', 0.040374815932701154),
  ('tasks', 0.038107980577363976),
  ('senior', 0.03553089734767573)],
 1: [('test', 0.09134018571500278),
  ('reporting', 0.03735850398026014),
  ('defect', 0.03411592614888728),
  ('traceability', 0.03318263840943188),
  ('devise', 0.031141705828414003),
  ('automation', 0.029437809491076303),
  ('engineer', 0

In [ ]:
model_serv.save("serv_60_5x10_")
topics_serv_all["60"] = reduced_s[0]
coherence_serv_all["60"] = get_coherence(model_serv,topics_serv_all["60"],cleaned_S)
print(coherence_serv_all)
reduced_s = model_serv.reduce_topics(cleaned_S,topics_serv_all["60"],nr_topics=30)

{'119': 0.7011363691472867, '60': 0.6633892266571741}


In [ ]:
model_serv.get_topic_info()

,Topic,Count,Name
0,-1,375,-1_deloitte_business_leadership_skills
1,0,221,0_requirements_servicenow_skills_knowledge
2,1,216,1_members_development_module_team
3,2,130,2_test_reporting_defect_traceability
4,3,123,3_software_rpa_engineering_java
5,4,115,4_software_engineering_ability_category
6,5,114,5_application_skills_nodejs_sitecore
7,6,94,6_solution_alternatives_solutions_problem
8,7,86,7_operating_intelligence_ai_business
9,8,83,8_network_cisco_routers_switches


In [ ]:
model_serv.get_topics()

{-1: [('deloitte', 0.01969981092810918),
  ('business', 0.013034126782238385),
  ('leadership', 0.011920059542094423),
  ('skills', 0.01110072183926311),
  ('services', 0.0109999505850094),
  ('development', 0.009850873276496195),
  ('management', 0.009772686638055679),
  ('clients', 0.009441654463092053),
  ('benefits', 0.008889059534926656),
  ('resources', 0.008838228629199165)],
 0: [('requirements', 0.030937599835470998),
  ('servicenow', 0.025510770042017097),
  ('skills', 0.017639278315570696),
  ('knowledge', 0.015335119526797428),
  ('management', 0.013240063238214568),
  ('clients', 0.012908169841239207),
  ('client', 0.012719534845895481),
  ('role', 0.012542848931617319),
  ('requirement', 0.011311890028525635),
  ('customer', 0.011246550012981102)],
 1: [('members', 0.07389985129577327),
  ('development', 0.06210297193697701),
  ('module', 0.06008456527333131),
  ('team', 0.05765215963382076),
  ('coach', 0.05141828776855439),
  ('member', 0.049847573302645054),
  ('mentor

In [ ]:
model_serv.visualize_hierarchy(height=800)

In [ ]:
model_serv.save("serv_30_5x10_")
topics_serv_all["30"] = reduced_s[0]
coherence_serv_all["30"] = get_coherence(model_serv,topics_serv_all["30"],cleaned_S)
print(coherence_serv_all)
reduced_s = model_serv.reduce_topics(cleaned_S,topics_serv_all["30"],nr_topics=15)

{'119': 0.7011363691472867, '60': 0.6633892266571741, '30': 0.5790222452634782}


In [ ]:
model_serv.get_topic_info()

,Topic,Count,Name
0,-1,593,-1_deloitte_skills_experience_management
1,0,321,0_experience_software_engineering_development
2,1,221,1_requirements_business_servicenow_skills
3,2,216,2_development_team_module_coach
4,3,181,3_azure_cloud_experience_aws
5,4,175,4_software_engineering_development_category
6,5,173,5_skills_application_requirements_nodejs
7,6,130,6_test_defect_automation_traceability
8,7,115,7_spark_cloud_hadoop_skills
9,8,98,8_data_sql_clients_analytics


In [ ]:
model_serv.get_topics()

{-1: [('deloitte', 0.026167018399099923),
  ('skills', 0.017910909445832058),
  ('experience', 0.01748911831977504),
  ('management', 0.015072748471752388),
  ('services', 0.014793256921590777),
  ('development', 0.014300917438371476),
  ('leadership', 0.013435647334109901),
  ('clients', 0.013324529057055499),
  ('professional', 0.013298706456874722),
  ('knowledge', 0.010929584436416754)],
 0: [('experience', 0.03038078083282375),
  ('software', 0.023714631998053085),
  ('engineering', 0.021952999610057818),
  ('development', 0.020923595002133023),
  ('honeywell', 0.01850899470264777),
  ('skills', 0.016523433542034987),
  ('automation', 0.01429630314963021),
  ('application', 0.012858913968723398),
  ('java', 0.012702163954050563),
  ('infrastructure', 0.011850996161051439)],
 1: [('requirements', 0.04188187765671432),
  ('business', 0.03421828455418018),
  ('servicenow', 0.029507089670683218),
  ('skills', 0.028019760006628758),
  ('experience', 0.022037854024828947),
  ('clients',

In [ ]:
model_serv.save("serv_15_5x10_")
topics_serv_all["15"] = reduced_s[0]
coherence_serv_all["15"] = get_coherence(model_serv,topics_serv_all["15"],cleaned_S)
print(coherence_serv_all)

{'119': 0.7011363691472867, '60': 0.6633892266571741, '30': 0.5790222452634782, '15': 0.6128979429040811}


In [ ]:
print(coherence_prod_all)

{'210': 0.6811005429005669, '60': 0.608197249102767, '30': 0.549071874062258, '15': 0.47400035526341305}


In [ ]:
print(coherence_ind_all)

{'300': 0.6717969094477065, '60': 0.6068783913475868, '30': 0.5582427828059361, '15': 0.44418823984965133}


# RESULTS

In [ ]:
from prettytable import PrettyTable 
  
myTable = PrettyTable(["Models vs Num_Topics","Default","60","30","15"]) 

temp = ["Industry"]
temp.extend(coherence_ind_all.values())
myTable.add_row(temp) 

temp = ["Product"]
temp.extend(coherence_prod_all.values())
myTable.add_row(temp) 

temp = ["Service"]
temp.extend(coherence_serv_all.values())
myTable.add_row(temp) 

print(myTable)

+----------------------+--------------------+--------------------+--------------------+---------------------+
| Models vs Num_Topics |      Default       |         60         |         30         |          15         |
+----------------------+--------------------+--------------------+--------------------+---------------------+
|       Industry       | 0.6717969094477065 | 0.6068783913475868 | 0.5582427828059361 | 0.44418823984965133 |
|       Product        | 0.6811005429005669 | 0.608197249102767  | 0.549071874062258  | 0.47400035526341305 |
|       Service        | 0.7011363691472867 | 0.6633892266571741 | 0.5790222452634782 |  0.6128979429040811 |
+----------------------+--------------------+--------------------+--------------------+---------------------+


In [ ]:
models = {}
models["60"]=BERTopic.load("Ind_60_5x10_")
models["30"]=BERTopic.load("Ind_30_5x10_")
models["15"]=BERTopic.load("ind_15_5x10_")

In [ ]:
models

{'15': <bertopic._bertopic.BERTopic at 0x7f756ba99690>,
 '30': <bertopic._bertopic.BERTopic at 0x7f756934f710>,
 '60': <bertopic._bertopic.BERTopic at 0x7f756c4be850>}

In [ ]:
with open("AllCompanies3.html", 'w') as plotfile:
    js = "<html><head><script src=\"plotly-2.4.2.min.js\"></script></head><body>"
    plotfile.write(js)
    plotfile.write("<h2>All Companies</h2>")
    
    for num_topics,model in models.items():
      plotfile.write("<h3>Number of Topics: "+num_topics+" </h3>")
      fig = model.visualize_hierarchy(height=1000)
      """
      plotfile.write('<div style="overflow-y: scroll; height:400px;">')
      for key,value in model.get_topics().items():
        plotfile.write('<p> Topic: '+str(key)+'</p><br><p>'+str(value)+"</p><br>")
      plotfile.write('</div>')
      """
      
      topics_list = topics_ind_all[num_topics]
      batch_size = 4
      plotfile.write('<div style="overflow-y: scroll; height:400px;">')

      for i in range(0, len(topics_list), batch_size):
        t = topics_list[i:i+batch_size]
        bar = model.visualize_barchart(topics=t,n_words=10,height=300,width=1200)
        plotfile.write(bar.to_html(full_html=False, include_plotlyjs=False))
      
      plotfile.write("</div")


    # more logic here   


      plotfile.write(fig.to_html(full_html=False, include_plotlyjs=False))

    #fig.write_image("uh.pdf",engine="kaleido")


      #plotfile.write("\n\n-------\n\n")  
  #files.download("60_chart_ind.html") 
files.download("AllCompanies3.html")

In [ ]:
with open("AllCountries.html", 'a') as plotfile:
  js = "<html><head><script src=\"plotly-2.4.2.min.js\"></script></head><body>"
  plotfile.write(js)
  #for t in topics_ind_all["60"]:
  fig = v_545.visualize_barchart(top_n_topics=20,n_words=20)
  #plotfile.write(fig.to_html(full_html=False, include_plotlyjs=False))
  fig.write_image("uh.pdf",engine="kaleido")


      #plotfile.write("\n\n-------\n\n")  
  #files.download("60_chart_ind.html") 
  files.download("uh.pdf")

# Syllabus Transform

In [ ]:
from google.colab import files

In [ ]:
!cp drive/MyDrive/syllabus.pickle .

In [ ]:
subs = []
f = open("syllabus.pickle", "rb")
subs = pickle.load(f)
f.close()

syllabus_list = [] #extracting lines
for des in subs:
    des = des.replace('.',',')
    sentences = des.split(".") 
    while '' in sentences:
        sentences.remove('')
    syllabus_list.extend(sentences)
    
    
print("Total sentences: ",len(syllabus_list)) 
    

In [ ]:
def first_option(model,syllabus_list,topics_s,filename):
  result_s = {}
  for x in range(0,len(syllabus_list)-1):
    if topics_s[x] != -1:
      if topics_s[x] not in result_s:
        result_s[topics_s[x]] = []
    
      result_s[topics_s[x]].append(syllabus_list[x])

  with open(filename, 'a') as plotfile:
    js = "<html><head><script src=\"plotly-2.4.2.min.js\"></script></head><body>"
    plotfile.write(js)
    for key, value in result_s.items():
      print(model.get_topic(key)) 
      print(value)
      plotfile.write(str(value))
      fig = model.visualize_barchart(topics=[key],n_words=10)
      #fig.show(renderer="colab")
      plotfile.write(fig.to_html(full_html=False, include_plotlyjs=False))
      plotfile.write("\n\n")

      print("\n\n")
  files.download(filename)  

In [ ]:
def second_option(model,syllabus_list,filename):
  with open(filename, 'a') as plotfile:
    result_hank = {}
    js = "<html><head><script src=\"plotly-2.4.2.min.js\"></script></head><body>"
    plotfile.write(js)
    for x in range(0,len(syllabus_list)-1):
      topics, similarity = model.find_topics(syllabus_list[x], top_n=3)
      print(syllabus_list[x],"\n")
      plotfile.write(syllabus_list[x]+"\n")
      print(topics,"\n")
      fig = model.visualize_barchart(topics=topics,n_words=10)
      #fig.show(renderer="colab")
      plotfile.write(fig.to_html(full_html=False, include_plotlyjs=False))
      print(similarity,"\n")
      plotfile.write(str(similarity))
      plotfile.write("\n\n-------\n\n")  
  files.download(filename) 

In [ ]:
second_option(model_serv,syllabus_list,"111JD_serv_55.html")

In [ ]:
second_option(model_prod,syllabus_list,"111JD_prod_55.html")

In [ ]:
second_option(model_ind,syllabus_list,"111JD_ind_55.html")